# Метод sort_values()


        by — имя или список имён столбцов, по значениям которых производится сортировка.
        axis — ось, по которой производится сортировка (0 — строки, 1 — столбцы). По умолчанию сортировка производится по строкам.
        ascending — сортировка по возрастанию (от меньшего к большему). По умолчанию параметр выставлен на True, для сортировки по убыванию (от большего к меньшему) необходимо выставить его на False.
        ignore_index — создаются ли новые индексы в таблице. По умолчанию выставлен на False и сохраняет индексы изначальной таблицы.
        inplace — производится ли замена исходной таблицы на отсортированную. По умолчанию параметр выставлен на False, то есть замены не производится. Чтобы переопределить исходную таблицу на отсортированную, необходимо выставить этот параметр на True.



# Метод groupby()


        by — имя или список имён столбцов, по которым производится группировка.
        axis — ось, по которой производится группировка (0 — строки, 1 — столбцы). По умолчанию группировка производится по строкам.
        as_index — добавляется ли дополнительный индекс к таблице. По умолчанию установлен на True.

## Метод groupby() возвращает объект DataFrameGroupBy, который хранит в себе информацию о том, какие строки относятся к определённой группе, и сам по себе не представляет для нас интереса. 
## Однако к этому объекту можно применять уже знакомые нам агрегирующие методы (mean, median, sum и т. д.), чтобы рассчитывать показатели внутри каждой группы.

# Метод pivot_table для построения сводных таблиц
## Основные параметры метода pivot_table()


        values — имя столбца, по которому необходимо получить сводные данные, применяя агрегирующую функцию;
        index — имя столбца, значения которого станут строками сводной таблицы;
        columns — имя столбца, значения которого станут столбцами сводной таблицы;
        aggfunc — имя или список имён агрегирующих функций (по умолчанию — подсчёт среднего, 'mean');
        fill_value — значение, которым необходимо заполнить пропуски (по умолчанию пропуски не заполняются).



## Доступ к данным в сводной таблице

In [15]:
pivot = melb_data_fe.pivot_table(
    values='Landsize', 
    index='Regionname', 
    columns='Type', 
    aggfunc=['median','mean'], 
    fill_value=0
)
pivot.columns

MultiIndex([('median',     'house'),
            ('median', 'townhouse'),
            ('median',      'unit'),
            (  'mean',     'house'),
            (  'mean', 'townhouse'),
            (  'mean',      'unit')],
           names=[None, 'Type'])

In [16]:
# Так, из таблицы pivot мы можем получить средние значения площадей участков для типа здания unit, просто последовательно обратившись по имени столбцов:
display(pivot['mean']['unit'])

Regionname
Eastern Metropolitan          330.444444
Eastern Victoria              295.333333
Northern Metropolitan         495.026538
Northern Victoria               0.000000
South-Eastern Metropolitan    357.864865
Southern Metropolitan         466.380245
Western Metropolitan          557.637232
Western Victoria                0.000000
Name: unit, dtype: float64

In [17]:
# Аналогично производится и фильтрация в данных. Например, если нам нужны регионы, в которых средняя площадь здания для домов типа house меньше их медианной площади, то мы можем найти их следующим образом:
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

median                       mean              \
Type                   house townhouse unit       house   townhouse   
Regionname                                                            
Southern Metropolitan  586.0     246.0    0  569.643881  278.858824   
Western Metropolitan   531.0     198.0   62  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

In [18]:
print(list(filtered_pivot.index))

['Southern Metropolitan', 'Western Metropolitan']


#  Объединение DataFrame: знакомимся с новыми данными

Итак, представим, что нам надо получить единую таблицу, в которой будут собраны рейтинги, даты выставления рейтингов, а также информация о фильмах. Вот как мы будем действовать:


Склеим таблицы ratings1 и ratings2 в единую структуру.


К полученной таблице с рейтингами присоединим столбец с датой выставления рейтинга.
	

Присоединим к нашей таблице информацию о названиях и жанрах фильмов.

In [2]:
import pandas as pd
ratings1 = pd.read_csv('data/ratings1.csv', sep=',')
ratings2 = pd.read_csv('data/ratings2.csv', sep=',')
dates = pd.read_csv('data/dates.csv', sep=',')
movies = pd.read_csv('data/movies.csv', sep=',')
display(movies.head())

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# Объединение DataFrame: concat

## Основные параметры метода concat()


        objs — список объектов DataFrame ([df1, df2,…]), которые должны быть сконкатенированы;
        axis — ось определяет направление конкатенации: 0 — конкатенация по строкам (по умолчанию), 1 — конкатенация по столбцам;
        join — либо inner (пересечение), либо outer (объединение); рассмотрим этот момент немного позже;
        ignore_index — по умолчанию установлено значение False, которое позволяет значениям индекса оставаться такими, какими они были в исходных данных. Если установлено значение True, параметр будет игнорировать исходные значения и повторно назначать значения индекса в последовательном порядке.



Примечание. Обратите внимание, что concat является функцией библиотеки, а не методом DataFrame. Поэтому её вызов осуществляется как pd.concat(...).

Для корректной конкатенации по строкам объединяемые таблицы должны иметь одинаковую структуру — идентичное число и имена столбцов.


In [4]:
# Итак, давайте склеим таблицы ratings1 и ratings2. Для этого передадим их в списке в функцию concat(). По умолчанию объединение происходит по строкам, что нам и нужно, поэтому параметр axis мы не трогаем.
ratings = pd.concat([ratings1,ratings2])
display(ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
60831,610,166534,4.0
60832,610,168248,5.0
60833,610,168250,5.0
60834,610,168252,5.0


In [5]:
# Это связано с тем, что по умолчанию concat сохраняет первоначальные индексы объединяемых таблиц, а обе наши таблицы индексировались, начиная от 0. Чтобы создать новые индексы, нужно выставить параметр ignore_index на True:
ratings = pd.concat(
    [ratings1, ratings2], 
    ignore_index=True
)
display(ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100832,610,166534,4.0
100833,610,168248,5.0
100834,610,168250,5.0
100835,610,168252,5.0


In [6]:
print('Число строк в таблице ratings: ', ratings.shape[0])
print('Число строк в таблице dates: ', dates.shape[0])
print(ratings.shape[0] == dates.shape[0])

Число строк в таблице ratings:  100837
Число строк в таблице dates:  100836
False


In [7]:
display(ratings1.tail(1))
display(ratings2.head(1))

,userId,movieId,rating
40000,274,5621,2.0


,userId,movieId,rating
0,274,5621,2.0


In [8]:
# Чтобы очистить таблицу от дублей, мы можем воспользоваться методом DataFrame drop_duplicates(), который удаляет повторяющиеся строки в таблице. Не забываем обновить индексы после удаления дублей, выставив параметр ignore_index в методе drop_duplicates() на значение True:
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])

Число строк в таблице ratings:  100836


In [9]:
# Наконец, мы можем добавить к нашей таблице с оценками даты их выставления. Для этого конкатенируем таблицы ratings и dates по столбцам:
ratings_dates = pd.concat([ratings, dates], axis=1)
display(ratings_dates.tail(7))

,userId,movieId,rating,date,year
100829,610,164179,5.0,2017-05-03 21:07:11,2017
100830,610,166528,4.0,2017-05-04 06:29:25,2017
100831,610,166534,4.0,2017-05-03 21:53:22,2017
100832,610,168248,5.0,2017-05-03 22:21:31,2017
100833,610,168250,5.0,2017-05-08 19:50:47,2017
100834,610,168252,5.0,2017-05-03 21:19:12,2017
100835,610,170875,3.0,2017-05-03 21:20:15,2017


# Объединение DataFrame: join, merge

## Основные параметры метода join()


        other — таблица, которую мы присоединяем. При объединении она является «правой», а исходная таблица, от имени которой вызывается метод, является «левой».
        how — параметр типа объединения. Он может принимать значения 'inner', 'left' (left outer), 'right' (right outer), и 'outer' (full outer). По умолчанию параметр установлен на 'left'.
        on — параметр, который определяет, по какому столбцу в «левой» таблице происходит объединение по индексам из «правой».
        lsuffix и rsuffix — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.



In [13]:
# Проверим это, объединив таблицы типом left. Так как в наших таблицах есть одноимённые столбцы, установим один из суффиксов, чтобы избежать ошибки:
joined_false = ratings_dates.join(
    movies,
    rsuffix='_right',
    how='left'
)
display(joined_false)

,userId,movieId,rating,date,year,movieId_right,title,genres
0,1,1,4.0,2000-07-30 18:45:03,2000,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2000,2.0,Jumanji (1995),Adventure|Children|Fantasy
2,1,6,4.0,2000-07-30 18:37:04,2000,3.0,Grumpier Old Men (1995),Comedy|Romance
3,1,47,5.0,2000-07-30 19:03:35,2000,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1,50,5.0,2000-07-30 18:48:51,2000,5.0,Father of the Bride Part II (1995),Comedy
...,...,...,...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,2017,NaN,NaN,NaN
100832,610,168248,5.0,2017-05-03 22:21:31,2017,NaN,NaN,NaN
100833,610,168250,5.0,2017-05-08 19:50:47,2017,NaN,NaN,NaN
100834,610,168252,5.0,2017-05-03 21:19:12,2017,NaN,NaN,NaN


In [21]:
# Однако это не тот результат, который мы хотели, ведь мы не получили соответствия фильмов и их рейтингов. Чтобы совместить таблицы по ключевому столбцу с помощью метода join(), необходимо использовать ключевой столбец в «правой» таблице в качестве индекса. Это можно сделать с помощью метода set_index(). Также необходимо указать название ключа в параметре on.
joined = ratings_dates.join(
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
display(joined.head())

,userId,movieId,rating,date,year,title,genres
0,1,1,4.0,2000-07-30 18:45:03,2000,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2000,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,2000,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,2000,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,2000,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


## Метод объединения merge
Аналогично предыдущему, метод merge() предназначен для слияния двух таблиц по ключевым столбцам или по индексам. Однако, в отличие от join(), метод merge() предлагает более гибкий способ управления объединением, благодаря чему является более популярным.


        right — присоединяемая таблица. По умолчанию она является «правой».
        how — параметр типа объединения. По умолчанию принимает значение 'inner'.
        on — параметр, который определяет, по какому столбцу происходит объединение. Определяется автоматически, но рекомендуется указывать вручную.
        left_on — если названия столбцов в «левой» и «правой» таблицах не совпадают, то данный параметр отвечает за наименования ключевого столбца исходной таблицы.
        right_on — аналогично предыдущему, параметр отвечает за наименование ключевого столбца присоединяемой таблицы.
        lsuffix и rsuffix — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.



In [23]:
# етод merge() в первую очередь предназначен для слияния таблиц по заданным ключам, поэтому он не требует установки ключевых столбцов в качестве индекса присоединяемой таблицы. Кроме того, данный метод позволяет объединять даже таблицы с разноимёнными ключами. Таким образом, merge() проще в использовании и более многофункционален, чем схожие методы.
merged = ratings_dates.merge(
    movies,
    on='movieId',
    how='left'
)
display(merged.head())

,userId,movieId,rating,date,year,title,genres
0,1,1,4.0,2000-07-30 18:45:03,2000,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2000,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,2000,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,2000,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,2000,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [24]:
print('Число строк в таблице ratings_dates: ', ratings_dates.shape[0])
print('Число строк в таблице merged: ', merged.shape[0])
print(ratings_dates.shape[0] == merged.shape[0])

Число строк в таблице ratings_dates:  100836
Число строк в таблице merged:  100836
True


## Особенности использования merge()

In [25]:
# Объединим ratings_dates с movies по ключевому столбцу movieId, но с параметром how='outer' (full outer) и выведем размер таблицы, а также её «хвост»:
merged2 = ratings_dates.merge(
    movies,
    on='movieId',
    how='outer'
)
print('Число строк в таблице merged2: ', merged2.shape[0])
display(merged2.tail())

Число строк в таблице merged2:  100854


,userId,movieId,rating,date,year,title,genres
100849,NaN,30892,NaN,NaT,NaN,In the Realms of the Unreal (2004),Animation|Documentary
100850,NaN,32160,NaN,NaT,NaN,Twentieth Century (1934),Comedy
100851,NaN,32371,NaN,NaT,NaN,Call Northside 777 (1948),Crime|Drama|Film-Noir
100852,NaN,34482,NaN,NaT,NaN,"Browning Version, The (1951)",Drama
100853,NaN,85565,NaN,NaT,NaN,Chalet Girl (2011),Comedy|Romance


In [27]:
# объединим таблицы ratings1 и ratings2, как мы уже делали раньше, но теперь используем метод merge():
merge_ratings = ratings1.merge(ratings2, how='outer')
print('Число строк в таблице merge_ratings: ', merge_ratings.shape[0])
display(merge_ratings)

Число строк в таблице merge_ratings:  100836


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


# 8. Закрепление знаний

In [1]:
import pandas as pd
ratings_movies = pd.read_csv('data/ratings_movies.csv', sep = ',')
display(ratings_movies.head(10))

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
5,5,1,70,3.0,2000-07-30 18:40:00,From Dusk Till Dawn (1996),Action|Comedy|Horror|Thriller
6,6,1,101,5.0,2000-07-30 18:14:28,Bottle Rocket (1996),Adventure|Comedy|Crime|Romance
7,7,1,110,4.0,2000-07-30 18:36:16,Braveheart (1995),Action|Drama|War
8,8,1,151,5.0,2000-07-30 19:07:21,Rob Roy (1995),Action|Drama|Romance|War
9,9,1,157,5.0,2000-07-30 19:08:20,Canadian Bacon (1995),Comedy|War


In [2]:
import re
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg)
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	    #очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None



In [6]:
# 8.1
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)
display(ratings_movies.head())

,Unnamed: 0,userId,movieId,rating,date,title,genres,year_release
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,1995.0
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,1995.0
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995.0
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995.0


In [7]:
ratings_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


In [9]:
# 8.2
mask = ratings_movies['year_release'] == 1999
ratings_movies[mask].groupby('title')['rating'].mean().sort_values()

title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

In [10]:
# 8.3
mask = ratings_movies['year_release'] == 2010
ratings_movies[mask].groupby('genres')['rating'].mean().sort_values()



genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

In [14]:
# 8.4
user = ratings_movies.groupby('userId')['genres'].nunique().sort_values(ascending=False)
display(user)

userId
599    524
414    482
448    403
380    399
474    395
      ... 
578     15
12      15
85      13
214     13
245     13
Name: genres, Length: 610, dtype: int64

In [17]:
# 8.5
user = ratings_movies.groupby('userId')['rating'].agg(
    ['count','mean'], 
).sort_values(['count','mean'], ascending=[True,False])
display(user)

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [23]:
# 8.6
mask1 = ratings_movies['year_release'] == 2018
grouped = ratings_movies[mask1].groupby('movieId')['rating'].agg(
    ['mean', 'count'],
)
grouped[grouped['count']>10].sort_values(
    by = ['mean','count'], 
    ascending=[False, False]
)

,mean,count
movieId,,
122912,4.000,13
187593,3.875,12


In [38]:

mask2= ratings_movies['movieId'] == 122912
ratings_movies[mask2]

,Unnamed: 0,userId,movieId,rating,date,title,genres,year_release
4031,4031,25,122912,5.0,2018-08-28 15:34:21,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi,2018.0
9105,9105,62,122912,4.0,2018-05-11 08:56:15,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi,2018.0
15233,15233,98,122912,5.0,2018-07-24 18:37:57,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi,2018.0
27192,27192,184,122912,5.0,2018-09-16 15:00:34,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi,2018.0
34894,34894,233,122912,2.0,2018-09-05 21:06:34,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi,2018.0
36357,36357,248,122912,4.0,2018-08-18 11:32:20,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi,2018.0
37347,37347,249,122912,4.5,2018-05-20 16:16:46,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi,2018.0
46571,46571,305,122912,4.5,2018-08-18 17:25:20,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi,2018.0
51989,51989,338,122912,1.5,2018-06-28 01:12:50,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi,2018.0
58020,58020,380,122912,5.0,2018-05-12 22:56:54,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi,2018.0


In [43]:
# 8.7
ratings_movies['date'] = pd.to_datetime(ratings_movies['date'])
ratings_movies['year_rating'] = ratings_movies['date'].dt.year


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Unnamed: 0    100836 non-null  int64         
 1   userId        100836 non-null  int64         
 2   movieId       100836 non-null  int64         
 3   rating        100836 non-null  float64       
 4   date          100836 non-null  datetime64[ns]
 5   title         100836 non-null  object        
 6   genres        100836 non-null  object        
 7   year_release  100818 non-null  float64       
 8   year_rating   100836 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 6.9+ MB


None

In [44]:
pivot = ratings_movies.pivot_table(
    values='rating',
    index='year_rating',
    columns='genres', 
    aggfunc='mean' 
    
)
display(pivot)



genres,(no genres listed),Action,Action|Adventure,Action|Adventure|Animation,Action|Adventure|Animation|Children,Action|Adventure|Animation|Children|Comedy,Action|Adventure|Animation|Children|Comedy|Fantasy,Action|Adventure|Animation|Children|Comedy|IMAX,Action|Adventure|Animation|Children|Comedy|Romance,Action|Adventure|Animation|Children|Comedy|Sci-Fi,...,Romance|Thriller,Romance|War,Romance|Western,Sci-Fi,Sci-Fi|IMAX,Sci-Fi|Thriller,Sci-Fi|Thriller|IMAX,Thriller,War,Western
year_rating,,,,,,,,,,,,,,,,,,,,,
1996,NaN,2.730769,3.454545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.666667,NaN,3.838095,NaN,3.117647
1997,NaN,3.538462,4.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.400000,NaN,3.923077,NaN,3.000000
1998,NaN,NaN,4.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.800000,NaN,NaN
1999,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.000,NaN,NaN,NaN,NaN,4.000000,NaN,3.700000,4.5,4.000000
2000,NaN,2.588235,3.738462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.000,NaN,3.0,3.416667,NaN,2.142857,NaN,3.087912,3.0,4.058824
2001,NaN,3.000000,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,2.500000,NaN,2.500000,NaN,3.477273,3.0,3.111111
2002,NaN,2.750000,4.304348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.000,NaN,NaN,3.750000,NaN,3.600000,NaN,3.583333,3.5,3.000000
2003,NaN,3.833333,3.277778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.375,2.5,NaN,2.333333,NaN,3.142857,NaN,3.250000,3.0,4.000000
2004,NaN,2.700000,4.136364,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,...,3.000,3.0,3.5,2.125000,NaN,NaN,NaN,3.464286,3.0,3.800000


In [67]:
# 8.8
orders = pd.read_csv('data/orders.csv', sep=';')
display(orders.head(5))
products = pd.read_csv('data/products.csv', sep=';')
display(products.head(5))

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5


,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR


In [74]:
orders_products = orders.merge(
    products, 
    left_on='ID товара',
    right_on='Product_ID',
    how='left')

display(orders_products.tail(5))

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
13,08.11.2019 08:36:21,2,7,Выполнен,Да,Нет,Да,47,1,47.0,Шатны Полосатый рейс,2999.0,RUR
14,08.11.2019 08:36:20,1,5,"Оплачен, формируется к отправке",Да,Нет,Нет,71,1,71.0,Платье Ночная Жизнь,7999.0,RUR
15,08.11.2019 08:36:20,1,5,"Оплачен, формируется к отправке",Да,Нет,Нет,86,1,86.0,"Носки Простые, муж",45.0,RUR
16,08.11.2019 08:36:20,1,5,"Оплачен, формируется к отправке",Да,Нет,Нет,51,1,51.0,Платье Аленький цветочек,4999.0,RUR
17,01.01.2001 00:00:00,0,1,"Оплачен, формируется к отправке",Да,Нет,Нет,666,1,NaN,NaN,NaN,NaN


In [75]:
# 8.9
mask= orders_products['Отменен'] == 'Да'
display(orders_products[mask].head(10))

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,124.0,Носки беговые Camino,999.0,RUR


In [80]:
# 8.10
orders_products['Profit'] = orders_products['Price'] * orders_products['Количество']
orders_products.groupby('ID Покупателя')['Price'].sum().sort_values(ascending=False)

ID Покупателя
7     17096.0
5     14042.0
8       697.0
1       448.0
9       344.0
10      199.0
Name: Price, dtype: float64